## Prediction of tropical storms intensity

In [2]:
# Packages loading
import os
import pandas as pd
import numpy as np

In [4]:
# Change current working directory
# current directory:
print(os.getcwd())
# set new directory
os.chdir('D:\_DSTI\A24_Python_Machine_learning\Project3_Tropical_storms')


D:\_DSTI\A24_Python_Machine_learning\Project3_Tropical_storms


# Dataset loading
IBTrACS dataset from NOAA.
International Best Track Archive for Climate Stewardship.
https://www.ncei.noaa.gov/products/international-best-track-archive


In [ ]:
# Dataset loading
df = pd.read_csv('.\ibtracs.csv', header=0) 
df = df.iloc[1:] # remove 2nd line (with index 1).
df = df.reset_index() #reset indexes ;
# Add a column "Index" corresponding to the previous indexes. Can be removed.
#storms.head()
#print(storms)
# storms: dataframe 297098 lines, 175 columns (174 + 1 for old indexes)

In [ ]:
# List of columns
columns_headers=list(df.columns)
print(columns_headers)

# PROBLEM: 
Empty cells are in fact filled with whitespace and thus NA are not properly detected

In [ ]:
df.isna().sum() # it seems that there is no NA, which is false.
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x) # remove whitespace from every cells
df.replace('', np.nan, inplace=True) # replace empty cells with NaN
df_NA = df.isna().sum() # it seems to work. 
# We can remove all columns filled only with NA

Data types:
When loading the table, because of the 2nd row of units, Python do not detect datatypes and everything is considered as "object".

In [ ]:
# convert data types : 
cols = df.filter(like="_WIND").columns
df[cols] = df[cols].apply(pd.to_numeric, errors="coerce")
df.dtypes

## Featuring engineering
Columns to remove:
    # index: column added by Python when updating indexes after removing the 2nd line.
    # NAME: same as SID except that some storms don't have name. Redundant info
    # NUMBER: number of the storm for the year. Restart at 1 each year. Not useful.
    # REUNION_R64_NE
    # REUNION_R64_SE
    # REUNION_R64_SW
    # REUNION_R64_NW
    # REUNION_GUST
    # TD9636_PRES
    # TD9635_WIND
    # TD9635_PRES
    # TD9635_ROCI
    # MLC_CLASS
    # MLC_WIND
    # MLC_PRES ----- All this previous columns are only empty cells
    # ALL _LAT: all latitude are more or less the same as in column LAT. Redundant info.
    # ALL _LON: all longitude are more or less the same as in column LON. Redundant info.
    # BASIN 
    # SUBBASIN: These two will probably be correlated to LAT and LON. Redundant info.
    # WMO_WIND: all data from agencies but differents units but no adjustement is made for differences in wind speed averaging. Data not comparable.
    # TD9636_WIND: in data description: "subjective, must be interpreted with caution". Not reliable.


Parameters common to all agencies: WIND and PRESS

In [ ]:
# Columns that are not useful/full of NA/not reliable:
STORMS = df.drop(columns=['index', 'NAME', 'NUMBER',
                                    'REUNION_R64_NE', 'REUNION_R64_SE', 'REUNION_R64_SW', 'REUNION_R64_NW', 
                                    'REUNION_GUST', 'TD9636_PRES', 'TD9635_WIND', 'TD9635_PRES', 'TD9635_ROCI', 'MLC_CLASS', 'MLC_WIND', "MLC_PRES",
                                    'BASIN', 'SUBBASIN',
                                    'WMO_WIND', 'TD9636_WIND',
                                    ])

# Latitude and longitude columns
We keep only LAT and LONG as it is the mean position (a combination of all the available positions)

In [ ]:
# Latitude and longitude columns
col_to_drop = STORMS.columns[STORMS.columns.str.contains('_LAT')] # all columns containing "_LAT"
STORMS.drop(col_to_drop, axis=1, inplace=True) # remove all columns with _LAT
col_to_drop = STORMS.columns[STORMS.columns.str.contains('_LON')] # all columns containing "_LON"
STORMS.drop(col_to_drop, axis=1, inplace=True) # remove all columns with _LON

# WIND columns
_WIND columsn from the different agencies have different units, and thus can not be gather together easily :
    # WMO_WIND: all data from agencies but differents units (no adjustement is made for differences in wind speed averaging)
    # I suggest to remove it. No reliable.
    # USA_WIND, DS824_WIND, NEUMANN_WIND, max speed 1-min averaged
    # CMA_WIND, max speed 2-min averaged
    # NEWDELHI_WIND, max speed 3-min averaged
    # TOKYO_WIND, HKO_WIND, KMA_WIND, REUNION_WIND, BOM_WIND, NADI_WIND, WELLINGTON_WIND max speed 10-min averaged
    # TD9636_WIND: in data description "estimates subjective, interpreted with caution". Remove it. 
    # TD9635_WIND, MLC_WIND: only NA

We can take USA_WIND as a base because it is where there is the less NAN and complete the missing data with other columns, especially DS824_WIND and NEUMANN_WIND as it is supposed to be the same unit (max speed 1-min averaged).
Problem: data are not in the same unit 
To convert from max speed 10-min averaged to 1-min averaged : * 1.12
IBTrACS technical details p5: 
    "The U.S. agencies (NOAA and JTWC) report a 1 min averaging time for the sustained (i.e. relatively long-lasting) winds. In most of the rest of the world, a 10 min averaging time is used for "sustained wind". It is possible to convert from peak 10 min wind to peak 1 min wind (roughly 12% higher for the latter) as a general rule."
It is not a very accurate method so I suggest not to use it too much.
(See part 6.2 Wind speed reporting differences from Technical details doc)

We create one column WIND to gather all wind data and one column AGENCY to track the origin of the data.

In [ ]:
# Create the two columns "WIND" and "AGENCY":
STORMS["WIND"] = np.nan
STORMS["WIND_AGENCY"] = np.nan

sources_adj = [
    ("USA_WIND", 1.0),  # No adjustment
    ("DS824_WIND", 1.0),  # No adjustment
    ("NEUMANN_WIND", 1.0),  # No adjustment
    ("TOKYO_WIND", 1.12),  # Adjustment factor
    ("HKO_WIND", 1.12),
    ("KMA_WIND", 1.12),
    ("REUNION_WIND", 1.12),
    ("BOM_WIND", 1.12),
    ("NADI_WIND", 1.12),
    ("WELLINGTON_WIND", 1.12)
]

# Iterate through the sources
for source, factor in sources_adj:
    mask = pd.isna(STORMS["WIND"]) & ~pd.isna(STORMS[source])  # Where WIND is NaN but source is not NaN
    STORMS.loc[mask, "WIND"] = STORMS.loc[mask, source] * factor
    STORMS.loc[mask, "WIND_AGENCY"] = source

In [ ]:
# we can remove all other _WIND columns
col_to_drop = STORMS.columns[STORMS.columns.str.contains('_WIND')] # all columns containing "_WIND"
STORMS.drop(col_to_drop, axis=1, inplace=True) # remove all columns with _WIND
STORMS.isna().sum()


## Detailed explaination for removing columns
# ----------------------------

In [ ]:
# List of storms
IDS = df.loc[:, ["SID", "NAME", "NUMBER"]] 
# SID and Name are the same except that some storms don't have names so let's keep SID and remove Name
# NUMBER is the number of the storm for the year. Restart at 1 each year. Remove it. 
IDS.describe() # There are 4767 different storms
IDS.isna().sum() # No NA in SID. 

In [ ]:
# LATITUDE
LAT = df.filter(like="LAT", axis=1) # All columns containing "LAT" in their headers
# All latitudes are the same most of the time (+- 0.1, sometimes more but mostly the same).
# All these columns contains the same information. 
LAT = LAT.loc[:, ['LAT']]
LAT.isna().sum() # No na in column LAT

In [ ]:
# LONGITUDE
LON = df.filter(like="LON", axis=1)
LON.isna().sum()
# same as with LAT, all longitudes are the same most of the time (+- 0.1, sometimes more but mostly the same).
# All these columns contains the same information. 
LON = LON.loc[:, ['LON']]
LON.isna().sum() # No na in column LAT

In [ ]:
# Missing data by location and parameters
STORMS_NA = STORMS.iloc[:, 13:122] # Extract columns with parameters per location
LOC_PARAM = [col.split("_", 1) 
         for col in STORMS_NA.columns] # split headers based on the first _ to split LOC and PARAM
LOC = [p[0] for p in LOC_PARAM] # extract Location from column names
LOC_unique = sorted(set(LOC)) # give a list of unique location
PARAM = [p[1] for p in LOC_PARAM] # extract parameters from column names
PARAM_unique = sorted(set(PARAM)) # give a list of unique parameters

NA = pd.DataFrame(0, index=LOC_unique, columns=PARAM_unique) # Prepare the dataframe to be filled with sum of na

for col in STORMS_NA.columns:
    LOC, PARAM = col.split('_', 1)  # Split only at the first underscore
    NA.loc[LOC, PARAM] += STORMS_NA[col].isna().sum() # sum of NA values
# Not very helpful

In [ ]:
# WIND
    # WMO_WIND: all data from agencies but differents units (no adjustement is made for differences in wind speed averaging)
    # I suggest to remove it. No reliable.
    # USA_WIND, DS824_WIND, NEUMANN_WIND, max speed 1-min averaged
    # CMA_WIND, max speed 2-min averaged
    # NEWDELHI_WIND, max speed 3-min averaged
    # TOKYO_WIND, HKO_WIND, KMA_WIND, REUNION_WIND, BOM_WIND, NADI_WIND, WELLINGTON_WIND max speed 10-min averaged
    # TD9636_WIND: in data description "estimates subjective, interpreted with caution". Remove it. 
    # TD9635_WIND, MLC_WIND: only NA
WIND = STORMS.filter(like="WIND", axis =1)
WIND.isna().sum()
WIND.dtypes
# convert wind to numeric
cols = WIND.columns
WIND[cols] = WIND[cols].apply(pd.to_numeric, errors="coerce")
    # We can take USA_WIND as a base because it is where there is the less NAN and complete the missing data with other columns, 
    # especially DS824_WIND and NEUMANN_WIND as it is supposed to be the same unit (max speed 1-min averaged).
"""
WIND.insert(0, "WIND", "")
WIND.insert(1, "AGENCY", "")

for i in WIND.index:
    if not pd.isna(WIND.at[i, "USA_WIND"]):
        WIND.at[i, "WIND"] = WIND.at[i, "USA_WIND"]
        WIND.at[i, "AGENCY"] = "USA_WIND" 
    elif pd.isna(WIND.at[i, "USA_WIND"]) and not pd.isna(WIND.at[i, "DS824_WIND"]):
        WIND.at[i, "WIND"] = WIND.at[i, "DS824_WIND"]
        WIND.at[i, "AGENCY"] = "DS824_WIND"
    elif pd.isna(WIND.at[i, "USA_WIND"]) and pd.isna(WIND.at[i, "DS824_WIND"]) and not pd.isna(WIND.at[i, "NEUMANN_WIND"]):
        WIND.at[i, "WIND"] = WIND.at[i, "NEUMANN_WIND"]
        WIND.at[i, "AGENCY"] = "NEUMANN_WIND"
    else:
        WIND.at[i, "WIND"] = np.nan
        WIND.at[i, "AGENCY"] = np.nan
WIND.isna().sum()
"""
# We don't fill much more data
# WIND.dropna(axis = 0, how='all', inplace = True) # inplace = True is to update the table
# WIND.isna().sum()
# When removing rows with only NA (thus with no data for wind) we still have 30080 rows with NA in WIND and with data somewherelse we could use.
# Problem: data are not in the same unit 
# To convert from max speed 10-min averaged to 1-min averaged : * 1.12
# IBTrACS technical details p5: 
    # "The U.S. agencies (NOAA and JTWC) report a 1 min averaging time for the sustained (i.e. relatively long-lasting) winds. 
    # In most of the rest of the world, a 10 min averaging time is used for "sustained wind". 
    # It is possible to convert from peak 10 min wind to peak 1 min wind (roughly 12% higher for the latter) as a general rule."
# It is not a very accurate method so I suggest not to use it too much.
# (See part 6.2 Wind speed reporting differences from Technical details doc)
"""
for i in WIND.index:
    if pd.isna(WIND.at[i, "USA_WIND"]) and not pd.isna(WIND.at[i, "TOKYO_WIND"]):
        WIND.at[i, "WIND"] = (WIND.at[i, "TOKYO_WIND"] * 1.12)
        WIND.at[i, "AGENCY"] = "TOKYO_WIND"
    elif pd.isna(WIND.at[i, "USA_WIND"]) and not pd.isna(WIND.at[i, "HKO_WIND"]):
        WIND.at[i, "WIND"] = (WIND.at[i, "HKO_WIND"] * 1.12)
        WIND.at[i, "AGENCY"] = "HKO_WIND"
    elif pd.isna(WIND.at[i, "USA_WIND"]) and not pd.isna(WIND.at[i, "KMA_WIND"]):
        WIND.at[i, "WIND"] = (WIND.at[i, "KMA_WIND"] * 1.12)
        WIND.at[i, "AGENCY"] = "KMA_WIND"
    elif pd.isna(WIND.at[i, "USA_WIND"]) and not pd.isna(WIND.at[i, "REUNION_WIND"]):
        WIND.at[i, "WIND"] = (WIND.at[i, "REUNION_WIND"] * 1.12)
        WIND.at[i, "AGENCY"] = "REUNION_WIND"
    elif pd.isna(WIND.at[i, "USA_WIND"]) and not pd.isna(WIND.at[i, "BOM_WIND"]):
        WIND.at[i, "WIND"] = (WIND.at[i, "BOM_WIND"] * 1.12)
        WIND.at[i, "AGENCY"] = "BOM_WIND"
    elif pd.isna(WIND.at[i, "USA_WIND"]) and not pd.isna(WIND.at[i, "NADI_WIND"]):
        WIND.at[i, "WIND"] = (WIND.at[i, "NADI_WIND"] * 1.12)
        WIND.at[i, "AGENCY"] = "NADI_WIND"
    elif pd.isna(WIND.at[i, "USA_WIND"]) and not pd.isna(WIND.at[i, "WELLINGTON_WIND"]):
        WIND.at[i, "WIND"] = (WIND.at[i, "WELLINGTON_WIND"] * 1.12)
        WIND.at[i, "AGENCY"] = "WELLINGTON_WIND" 
WIND.isna().sum() 
"""

# Or in a cleaner way:
sources_adj = [
    ("USA_WIND", 1.0),  # No adjustment
    ("DS824_WIND", 1.0),  # No adjustment
    ("NEUMANN_WIND", 1.0),  # No adjustment
    ("TOKYO_WIND", 1.12),  # Adjustment factor
    ("HKO_WIND", 1.12),
    ("KMA_WIND", 1.12),
    ("REUNION_WIND", 1.12),
    ("BOM_WIND", 1.12),
    ("NADI_WIND", 1.12),
    ("WELLINGTON_WIND", 1.12)
]

# Create the two columns "WIND" and "AGENCY:
WIND["WIND"] = np.nan
WIND["WIND_AGENCY"] = np.nan

# Iterate through the sources
for source, factor in sources_adj:
    mask = pd.isna(WIND["WIND"]) & ~pd.isna(WIND[source])  # Where WIND is NaN but source is not NaN
    WIND.loc[mask, "WIND"] = WIND.loc[mask, source] * factor
    WIND.loc[mask, "WIND_AGENCY"] = source
WIND.dropna(axis = 0, how='all', inplace = True)
WIND.isna().sum()



C:\Users\Claire\AppData\Local\Temp\ipykernel_6364\535304703.py:4: DtypeWarning: Columns (1,2,8,9,14,19,20,172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('.\ibtracs.csv', header=0)


['index', 'SID', 'SEASON', 'NUMBER', 'BASIN', 'SUBBASIN', 'NAME', 'ISO_TIME', 'NATURE', 'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'WMO_AGENCY', 'TRACK_TYPE', 'DIST2LAND', 'LANDFALL', 'IFLAG', 'USA_AGENCY', 'USA_ATCF_ID', 'USA_LAT', 'USA_LON', 'USA_RECORD', 'USA_STATUS', 'USA_WIND', 'USA_PRES', 'USA_SSHS', 'USA_R34_NE', 'USA_R34_SE', 'USA_R34_SW', 'USA_R34_NW', 'USA_R50_NE', 'USA_R50_SE', 'USA_R50_SW', 'USA_R50_NW', 'USA_R64_NE', 'USA_R64_SE', 'USA_R64_SW', 'USA_R64_NW', 'USA_POCI', 'USA_ROCI', 'USA_RMW', 'USA_EYE', 'TOKYO_LAT', 'TOKYO_LON', 'TOKYO_GRADE', 'TOKYO_WIND', 'TOKYO_PRES', 'TOKYO_R50_DIR', 'TOKYO_R50_LONG', 'TOKYO_R50_SHORT', 'TOKYO_R30_DIR', 'TOKYO_R30_LONG', 'TOKYO_R30_SHORT', 'TOKYO_LAND', 'CMA_LAT', 'CMA_LON', 'CMA_CAT', 'CMA_WIND', 'CMA_PRES', 'HKO_LAT', 'HKO_LON', 'HKO_CAT', 'HKO_WIND', 'HKO_PRES', 'KMA_LAT', 'KMA_LON', 'KMA_CAT', 'KMA_WIND', 'KMA_PRES', 'KMA_R50_DIR', 'KMA_R50_LONG', 'KMA_R50_SHORT', 'KMA_R30_DIR', 'KMA_R30_LONG', 'KMA_R30_SHORT', 'NEWDELHI_LAT', 'NEWDELHI

C:\Users\Claire\AppData\Local\Temp\ipykernel_6364\535304703.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x) # remove whitespace from every cells
C:\Users\Claire\AppData\Local\Temp\ipykernel_6364\535304703.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('', np.nan, inplace=True) # replace empty cells with NaN
C:\Users\Claire\AppData\Local\Temp\ipykernel_6364\535304703.py:82: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = fr